In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-data/cleaned (1).csv
/kaggle/input/data-storm-5-0/train_kaggle.csv
/kaggle/input/data-storm-5-0/test_kaggle.csv


In [2]:
clean_set = pd.read_csv('/kaggle/input/cleaned-data/cleaned (1).csv')

In [3]:
clean_set.head()

,Unnamed: 0,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,total_sales,cluster_city_enco,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,131072,10370993.0,Moratuwa,2072.47,1462.92,8046.06,4.0,11581.45,3,0.18,0.13,0.69
1,131073,10751334.0,Nuwara Eliya,1500.60,1200.48,6502.60,4.0,9203.68,4,0.16,0.13,0.71
2,131074,10482183.0,Dehiwala-Mount Lavinia,2862.00,1431.00,9444.60,4.0,13737.60,2,0.21,0.10,0.69
3,131075,10594832.0,Kelaniya,2103.15,1822.73,9394.07,4.0,13319.95,2,0.16,0.14,0.71
4,131076,10418774.0,Peliyagoda,1115.46,619.70,4275.93,4.0,6011.09,2,0.19,0.10,0.71


In [4]:
clean_set = clean_set.drop(columns=['Customer_ID', 'luxury_sales', 'fresh_sales', 'total_sales', 'cluster_city_enco', 'dry_sales'])

In [5]:
clean_set = clean_set.drop(columns=[clean_set.columns[0]])

In [6]:
clean_set.head()

,outlet_city,cluster_catgeory,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,Moratuwa,4.0,0.18,0.13,0.69
1,Nuwara Eliya,4.0,0.16,0.13,0.71
2,Dehiwala-Mount Lavinia,4.0,0.21,0.10,0.69
3,Kelaniya,4.0,0.16,0.14,0.71
4,Peliyagoda,4.0,0.19,0.10,0.71


In [7]:
# test set
test = pd.read_csv('/kaggle/input/data-storm-5-0/test_kaggle.csv')

In [8]:
test.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
0,33574,batticaloa,2686.5,3582,12537
1,10089,batticaloa,1717.56,2576.34,9446.58
2,38329,batticaloa,854.04,1242.24,5201.88
3,11376,batticaloa,1638.12,2320.67,9282.68
4,12410,batticaloa,1039.09,1518.67,5435.24


In [9]:
clean_set['cluster_catgeory'] = clean_set['cluster_catgeory'].astype(int)

In [10]:
label_mapping = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}

clean_set['cluster_catgeory'] = clean_set['cluster_catgeory'].replace(label_mapping)

In [11]:
clean_set = clean_set.drop(columns=['outlet_city'])

In [12]:
from sklearn.model_selection import train_test_split
X = clean_set.drop(columns=['cluster_catgeory'])
y = clean_set['cluster_catgeory']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
X

,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,0.18,0.13,0.69
1,0.16,0.13,0.71
2,0.21,0.10,0.69
3,0.16,0.14,0.71
4,0.19,0.10,0.71
...,...,...,...
612077,0.33,0.34,0.32
612078,0.33,0.32,0.35
612079,0.33,0.32,0.34
612080,0.33,0.31,0.36


In [14]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm



# Identify categorical features
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']

# Convert categorical features to category data type
for feature in categorical_features:
    X_train[feature] = X_train[feature].astype('category')
    X_val[feature] = X_val[feature].astype('category')

# Define hyperparameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'max_depth': -1,
    'random_state': 42
}

# Train the model with early stopping and optimized hyperparameters
lgbm_model = LGBMRegressor(**params)
lgbm_model.fit(X_train, y_train, eval_set=[(X_val, y_val)])


# Make predictions on the validation set
val_preds = lgbm_model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, val_preds)
print(f'Validation MSE: {mse:.4f}')

Validation MSE: 0.1386


In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [16]:
val_preds = np.round(val_preds).astype(int)
# Evaluate the final model
accuracy = accuracy_score(y_val, val_preds)
f1 = f1_score(y_val, val_preds, average='macro')
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation F1-score (macro): {f1:.4f}')
micro_prec = precision_score(y_val, val_preds, average='micro')
print(f"Micro-averaged Precision: {micro_prec:.4f}")

Validation Accuracy: 0.8892
Validation F1-score (macro): 0.5893
Micro-averaged Precision: 0.8892


In [17]:
from catboost import CatBoostClassifier, Pool



# Identify categorical features
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']

# Create Pool objects
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)

# Set parameters
params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6,
    'l2_leaf_reg': 3,
    'random_strength': 1,
    'bagging_temperature': 0.5,
    'metric_period': 50,
    'od_type': 'Iter',
    'od_wait': 50,
    'allow_writing_files': False
}

# Train the model
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=val_pool, use_best_model=True, verbose=100)

# Make predictions on the validation set
val_preds = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, val_preds)
print(f'Validation MSE: {mse:.4f}')

# Evaluate the final model
accuracy = accuracy_score(y_val, val_preds)
f1 = f1_score(y_val, val_preds, average='macro')
print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation F1-score (macro): {f1:.4f}')
micro_prec = precision_score(y_val, val_preds, average='micro')
print(f"Micro-averaged Precision: {micro_prec:.4f}")

0:	learn: 1.3532878	test: 1.3536371	best: 1.3536371 (0)	total: 270ms	remaining: 4m 29s
100:	learn: 0.0723613	test: 0.0731534	best: 0.0731534 (100)	total: 14.6s	remaining: 2m 9s
200:	learn: 0.0720494	test: 0.0730360	best: 0.0730186 (178)	total: 28.2s	remaining: 1m 52s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.07301860086
bestIteration = 178

Shrink model to first 179 iterations.
Validation MSE: 0.2209
Validation Accuracy: 0.9534
Validation F1-score (macro): 0.7865
Micro-averaged Precision: 0.9534


In [18]:
def convert_to_float_and_find_errors(dataframe, column):
    error_rows = []
    for index, value in dataframe[column].items():
        try:
            float(value)
        except ValueError:
            error_rows.append((index, value))
    return error_rows

# Convert 'luxury_sales' column to float and find rows causing ValueError
error_rows = convert_to_float_and_find_errors(test, 'luxury_sales')

# Print rows causing ValueError
print("Rows causing ValueError:")
for index, value in error_rows:
    print(f"Index: {index}, Value: {value}")

Rows causing ValueError:
Index: 6, Value: Seven Hundread and sixty Ruppees
Index: 6299, Value: Eight Hundread and fifteen ruppes


In [19]:
test['luxury_sales'][6] = 760.0 
test['luxury_sales'][6299] = 850.0

/tmp/ipykernel_18/2255363823.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['luxury_sales'][6] = 760.0
/tmp/ipykernel_18/2255363823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

In [20]:
# Convert 'luxury_sales' column to float and find rows causing ValueError
error_rows = convert_to_float_and_find_errors(test, 'fresh_sales')

# Print rows causing ValueError
print("Rows causing ValueError:")
for index, value in error_rows:
    print(f"Index: {index}, Value: {value}")

Rows causing ValueError:
Index: 13949, Value: Nine thousand ruppess


In [21]:
test['fresh_sales'][13949] = 9000.0

/tmp/ipykernel_18/1902044106.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['fresh_sales'][13949] = 9000.0
/tmp/ipykernel_18/1902044106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [22]:
# Convert 'luxury_sales' column to float and find rows causing ValueError
error_rows = convert_to_float_and_find_errors(test, 'dry_sales')

# Print rows causing ValueError
print("Rows causing ValueError:")
for index, value in error_rows:
    print(f"Index: {index}, Value: {value}")

Rows causing ValueError:
Index: 40746, Value: Three Thousand seven hundread ruppees


In [23]:
test['dry_sales'][40746] = 3700.0

/tmp/ipykernel_18/807833885.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['dry_sales'][40746] = 3700.0
/tmp/ipykernel_18/807833885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

In [24]:
test['luxury_sales'] = test['luxury_sales'].astype(float)
test['fresh_sales'] = test['fresh_sales'].astype(float)
test['dry_sales'] = test['dry_sales'].astype(float)

In [25]:
test['total_sales'] = test['luxury_sales'] + test['fresh_sales'] + test['dry_sales']
test['total_sales'].describe().round(2)

count    40749.00
mean     11061.55
std       3832.32
min       4603.68
25%       7861.56
50%      10801.00
75%      13748.40
max      20784.40
Name: total_sales, dtype: float64

In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40749 entries, 0 to 40748
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Customer_ID   40749 non-null  int64  
 1   outlet_city   40749 non-null  object 
 2   luxury_sales  40749 non-null  float64
 3   fresh_sales   40749 non-null  float64
 4   dry_sales     40749 non-null  float64
 5   total_sales   40749 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.9+ MB


In [27]:
test['luxury_sales'] = (test['luxury_sales'] / test['total_sales']).round(2)  # Round to four decimal places
test['fresh_sales'] = (test['fresh_sales'] / test['total_sales']).round(2)
test['dry_sales'] = (test['dry_sales'] / test['total_sales']).round(2)


In [28]:
clean_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612082 entries, 0 to 612081
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cluster_catgeory    612082 non-null  int64  
 1   luxery_sales_ratio  612082 non-null  float64
 2   fresh_sales_ratio   612082 non-null  float64
 3   dry_sales_ratio     612082 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 18.7 MB


In [29]:
Customer_ID = test[['Customer_ID']]

In [30]:
test['luxery_sales_ratio'] = test['luxury_sales']
test['fresh_sales_ratio'] = test['fresh_sales']
test['dry_sales_ratio'] = test['dry_sales']

In [31]:
Customer_ID

,Customer_ID
0,33574
1,10089
2,38329
3,11376
4,12410
...,...
40744,33698
40745,4185
40746,28664
40747,11874


In [32]:
X = test.drop(columns=['Customer_ID', 'total_sales', 'outlet_city', 'luxury_sales', 'fresh_sales', 'dry_sales'])


In [33]:
X

,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,0.14,0.19,0.67
1,0.12,0.19,0.69
2,0.12,0.17,0.71
3,0.12,0.18,0.70
4,0.13,0.19,0.68
...,...,...,...
40744,0.33,0.35,0.33
40745,0.34,0.32,0.33
40746,0.33,0.34,0.32
40747,0.34,0.33,0.34


In [34]:
# Identify categorical features
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']

# Convert categorical features to category data type
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

In [35]:
X.head()

,luxery_sales_ratio,fresh_sales_ratio,dry_sales_ratio
0,0.14,0.19,0.67
1,0.12,0.19,0.69
2,0.12,0.17,0.71
3,0.12,0.18,0.70
4,0.13,0.19,0.68


In [36]:
# Make predictions on the validation set
val_preds = model.predict(X)

In [37]:
val_preds

array([[0],
       [0],
       [0],
       ...,
       [4],
       [4],
       [4]])

In [38]:
val_preds = np.round(val_preds).astype(int)

In [39]:
val_preds

array([[0],
       [0],
       [0],
       ...,
       [4],
       [4],
       [4]])

In [40]:
array_df = pd.DataFrame(val_preds, columns=['cluster category'])

In [41]:
array_df['cluster category'].unique()

array([0, 5, 4, 2, 1, 3])

In [42]:
label_unmapping = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6}

array_df['cluster category'] = array_df['cluster category'].replace(label_unmapping)

In [43]:
submission_1 = pd.concat([Customer_ID, array_df], axis=1)

In [44]:
submission_1.to_csv('cat_output_file.csv', index=False)